# Train DSF e iESPnet

In [18]:
import sys
import os
import torch

import pandas as pd

sys.path.append(os.path.abspath(os.path.join('..','..','iESPnet_SRC_main','utilities')))
from Generator         import SeizureDatasetLabelTimev2, scale_spec, permute_spec, smoothing_label
from Model             import iESPnet
from TrainEval         import train_model_opt, test_model, train_model, get_thr_output, get_performance_indices

sys.path.append(os.path.abspath(os.path.join('..','05-Train-Test')))
from utilit_train_test import make_weights_for_balanced_classes

In [ ]:
torch.manual_seed(0)

In [6]:
# direccion donde se encuentran los espectrogramas 

SPE_DIR        = '/media/martin/Disco2/Rns_Data/PITT_PI_EEG/'
meta_data_file = '/media/martin/Disco2/Rns_Data/PITT_PI_EEG/METADATA/allfiles_metadata.csv'

df_meta        = pd.read_csv(meta_data_file)

In [10]:
FREQ_MASK_PARAM = 10
TIME_MASK_PARAN = 20
N_CLASSES       = 1
learning_rate   = 1e-3
batch_size      = 128
epochs          = 20
num_workers     = 4


save_path       = 'models_DSF_iESPnet/'
patients        = df_meta['rns_id'].unique().tolist()

In [11]:
hparams = {
        "n_cnn_layers" : 3,
        "n_rnn_layers" : 3,
        "rnn_dim"      : [150, 100, 50],
        "n_class"      : N_CLASSES,
        "out_ch"       : [8,8,16],
        "dropout"      : 0.3,
        "learning_rate": learning_rate,
        "batch_size"   : batch_size,
        "num_workers"  : num_workers,
        "epochs"       : epochs
        }

In [15]:
# ejemplo para un unico paciente s = 0 --- patient = PIT-RNS1603

s = 0

In [ ]:
model = iESPnet(
                hparams['n_cnn_layers'],
                hparams['n_rnn_layers'],
                hparams['rnn_dim'],
                hparams['n_class'],
                hparams['out_ch'],
                hparams['dropout'],
               )

In [16]:
save_runs        = save_path + patients[s] + '/runs/'
save_models      = save_path + patients[s] + '/models/'
save_predictions = save_path + patients[s] + '/results/'
save_figs        = save_path + patients[s] + '/figs/'

if not os.path.exists(save_path):
    os.makedirs(save_path)
    
if not os.path.exists(save_runs):
    os.makedirs(save_runs)
    
if not os.path.exists(save_models):
    os.makedirs(save_models)
    
if not os.path.exists(save_predictions):
    os.makedirs(save_predictions)
    
if not os.path.exists(save_figs):
    os.makedirs(save_figs)

print('Running training for subject ' + patients[s] + ' [s]: ' + str(s))

Running training for subject PIT-RNS1603 [s]: 0


In [17]:
# define train y test de df_meta

train_df = df_meta.copy()
test_df  = df_meta[df_meta['rns_id'] == patients[s]]
test_df.reset_index(drop=True, inplace=True)
train_df.drop(train_df[train_df['rns_id'] == patients[s]].index, inplace = True)

In [19]:
# Dataloaders creados

train_data_ori = SeizureDatasetLabelTimev2(
                                           file=train_df,
                                           root_dir=SPE_DIR,
                                           transform=None, 
                                           target_transform=smoothing_label(),
                                          )